
# Задание N 05. Исследование зависимости светимости от температуры для абсолютно черного тела.

Абсолютно черное тело излучает энергию пропорционально четвертой степени температуры
$$ E = 36.9 * 10^{-12} T^4 $$
(E-мощность излучения в вт/см, Т-температура в градусах Кельвина).
Часть общей энергии, заключенная в видимом спектре частот, с длиной волны от $\lambda_1$ до $\lambda_2$ находится интегрированием уравнения Планка:

$$
E_{видимая} = E_{в} = \int\limits_{\lambda_1}^{\lambda_2} 
\frac{2.39 * 10^{-11}dx}{x^{5}(e^{\frac{1.432}{Tx}} - 1)} 
$$



Тогда светимость (в процентах) рассчитывается по формуле:
$$
EFF = \frac{E_{в}}{E} * 100\% = \frac{64.77}{T^4} * \int\limits_{\lambda_1}^{\lambda_2} 
\frac{dx}{x^{5}(e^{\frac{1.432}{Tx}} - 1)} 
$$


$$
\lambda_1 = y * 31.66675 * 10^{-5},
        где\ y\ -\ корень\ уравнения:
        \\2 * \sqrt{x} = \cos{\frac{\pi*x}{2}}
        \\
        y = x = 0.221050639495735\ =>
        \\ \lambda_1 = 6.999955338251567 * 10^{-5}
$$



$$
\lambda_2 = - z * 3.039830 * 10^{-5},
\\ где\ z\ -\ значение,\ минимизирующее\ f(z)\ на\ [-2, -1]
        \\ f(z) = (e^z)*(2*z^{2}-4) + (2*z^{2}-1)^{2} + e^{2*z} - 3*z^{4};\ z = -1.3159745 =>
        \\ \lambda_2 = 4.000338764335*10^{-5}
$$


#### Вычислить EFF в диапазоне температур от Т=1000 К до Т=9000 К с шагом 1000 К.

**По полученным данным построить график светимости от
температуры, взяв достаточное количество точек.**

**Оценить погрешность результата и влияние на
точность погрешности в задании $\lambda_1$ и $\lambda_2$**

In [1]:
"""
lambda1 = y * 31.66675 * 10**(-5),
    где y - корень уравнения:
    2 * sqrt(x) = cos((PI*x)/2)
    x ~= 0.221050639495735

lambda2 = -z * 3.039830 * 10**(-5),
    где z - значение, минимизирующее f(z) на [-2, -1]
    f(z) = (e**z)*(2*z**2-4) + (2*z**2-1)**2 + e**(2*z) - 3*z**4


EFF = (E_в / E) * 100% =
64.77 / (T**4) * (integrate from lambda1 to lambda 2:
    dx / ((x**5) * (e**(1.432/(Tx)) - 1) )
"""
print("We Begin")

We Begin


Напишем функции для нахождения x и z

In [2]:
import scipy.optimize as optimize
from scipy.integrate import quad
import numpy as np
from math import sqrt, cos, pi

accuracy = 1e-10


def fun_x(x):
    return 2 * sqrt(x[0]) - cos((pi * x[0]) / 2)


def fun_z(z):
    return (np.e ** z) * (2 * z ** 2 - 4) + (2 * z ** 2 - 1) ** 2 + np.e ** (2 * z) - 3 * z ** 4


Найдем х используя метод fsolve

In [3]:

res_x = optimize.fsolve(fun_x, np.array([0]))
print(res_x)
# [0.22105064]

[0.22105064]


Найдем значение z, минимизирующее f(z) на промежутке [-2,-1]

In [4]:
lower_bound = -2
upper_bound = -1
bounds = optimize.Bounds(lower_bound, upper_bound)
res_z = optimize.minimize(fun_z,
                          np.array((lower_bound + upper_bound) / 2),
                          bounds=bounds,
                          method='TNC',
                          tol=accuracy).x
print(res_z)
# [-1.3159745]

[-1.3159745]


Вычислим полученные лямбды по формулам

In [5]:
lambda_1 = res_x[0] * 31.66675 * 10 ** (-5)
lambda_2 = -res_z[0] * 3.039830 * 10 ** (-5)
print(f'lambda_1 = {lambda_1}\nlambda_2 = {lambda_2}') \
    # lambda_1 = 6.999955338251561e-05
# lambda_2 = 4.00033875088364e-05

lambda_1 = 6.999955338251561e-05
lambda_2 = 4.00033875088364e-05


Запишем подынтегральное выражение в виде функции integrand и воспользуемся ей для вычисления EFF.
В данном случае нижняя граница интегрирования (l_bound) = $\lambda_1$, верхняя (u_bound) = $\lambda_2$.
T - аргумент подынтегрального выражения

In [6]:
def integrand(x, T):
    return 1 / ((x ** 5) * (np.exp(1.432 / (T * x)) - 1))


def EFF(T, l_bound, u_bound):
    return (
            64.77 / (T ** 4) * quad(integrand, l_bound, u_bound, args=T, epsabs=accuracy)[0]
    )

In [7]:
## Давайте протестируем взятие интеграла.
T = 1000.0
print(quad(integrand, lambda_1, lambda_2, args=T, epsabs=accuracy)[0])
# -3085873.2324665096 - вычисленный интеграл в WolframAlpha

-3085873.2324665114


In [8]:
ans = []
T0 = 1000
step = 1000
for i in range(9):
    T = T0 + i * step
    print(f"T = {T}")

    t_res = EFF(T, lambda_1, lambda_2)
    ans.append(t_res)
    print(f'EFF_{i} = {t_res}')

T = 1000
EFF_0 = -0.00019987200926685595
T = 2000
EFF_1 = -0.8057657916443762
T = 3000
EFF_2 = -8.254919402732305
T = 4000
EFF_3 = -21.060996692103878
T = 5000
EFF_4 = -31.821521264911222
T = 6000
EFF_5 = -37.679683945031
T = 7000
EFF_6 = -39.30610173925311
T = 8000
EFF_7 = -38.22217001451057
T = 9000
EFF_8 = -35.69935643933249
